In [4]:
import rk_solver_cpp
from scipy.integrate import ode
import SundialsPy as SP
import numpy as np
import cantera as ct
import matplotlib.pyplot as plt
import pandas as pd
from typing import Tuple, List, Dict, Any, Optional
import os
from datetime import datetime
import time
from tqdm import tqdm
from utils import *

In [5]:
fuel = 'nc12h26:1.0'
oxidizer = 'N2:3.76, O2:1.0'
fuel_species = 'nc12h26'
mechanism_file = '/Users/elotech/Downloads/research_code/large_mechanism/n-dodecane.yaml'
rtol = 1e-6
atol = 1e-8

In [6]:
def reset_gas(temperature=600, pressure=101325, phi=1):
    gas = ct.Solution(mechanism_file)
    gas.set_equivalence_ratio(phi, fuel, 'O2:1, N2:3.76')
    gas.TPX = temperature, pressure, gas.X
    y0 = get_initial_state(gas)
    return gas, y0

In [7]:
implicit_solvers = [SP.arkode.ButcherTable.ARK2_DIRK_3_1_2, SP.arkode.ButcherTable.ESDIRK325L2SA_5_2_3, 
                    SP.arkode.ButcherTable.TRBDF2_3_3_2, SP.arkode.ButcherTable.ESDIRK436L2SA_6_3_4, SP.arkode.ButcherTable.ESDIRK43I6L2SA_6_3_4, 
                    SP.arkode.ButcherTable.QESDIRK436L2SA_6_3_4, 
                    SP.arkode.ButcherTable.CASH_5_2_4, SP.arkode.ButcherTable.CASH_5_3_4, SP.arkode.ButcherTable.SDIRK_5_3_4, 
                    SP.arkode.ButcherTable.ARK436L2SA_DIRK_6_3_4, SP.arkode.ButcherTable.ESDIRK437L2SA_7_3_4, SP.arkode.ButcherTable.ARK437L2SA_DIRK_7_3_4]

print(f"Number of implicit solvers = {len(implicit_solvers)}")

explicit_solvers = [SP.arkode.ButcherTable.HEUN_EULER_2_1_2 , SP.arkode.ButcherTable.BOGACKI_SHAMPINE_4_2_3,
        SP.arkode.ButcherTable.ARK324L2SA_ERK_4_2_3, SP.arkode.ButcherTable.ZONNEVELD_5_3_4,
        SP.arkode.ButcherTable.ARK436L2SA_ERK_6_3_4, SP.arkode.ButcherTable.ARK437L2SA_ERK_7_3_4,
        SP.arkode.ButcherTable.ARK548L2SA_ERK_8_4_5,
        SP.arkode.ButcherTable.VERNER_8_5_6,
        SP.arkode.ButcherTable.FEHLBERG_13_7_8]

print(f"Number of explicit solvers = {len(explicit_solvers)}")

Number of implicit solvers = 12
Number of explicit solvers = 9


In [8]:
solver_to_run = [SP.arkode.ButcherTable.ARK2_DIRK_3_1_2, SP.arkode.ButcherTable.TRBDF2_3_3_2, SP.arkode.ButcherTable.HEUN_EULER_2_1_2 , SP.arkode.ButcherTable.BOGACKI_SHAMPINE_4_2_3]

print(f"Number of solvers to plot = {len(solver_to_run)}")

Number of solvers to plot = 4


In [9]:
temperature = 800
pressure = 101325
phi = 1
gas, y0 = reset_gas(temperature, pressure, phi)
t0 = 0.0
end_time = 8e-2
timestep = 1e-6
species_to_track = gas.species_names
fuel = 'nc12h26'
time_limit = 240.0
table_id = SP.arkode.ButcherTable.HEUN_EULER_2_1_2


In [ ]:
# Define parameters for different runs
run_params = [
    ('reference', 1e-12, 1e-10),
    ('bdf_results_high', 1e-8, 1e-10), 
    ('bdf_results_low', 1e-6, 1e-8)
]

bdf_results = {}

# Loop through parameters and run experiments
for result_name, rtol, atol in run_params:
    gas, y0 = reset_gas(temperature, pressure, phi)
    method = 'cvode_bdf'
    
    results = run_integration_experiment(
                method, gas, y0, t0, end_time, timestep,
                rtol, atol, species_to_track,
                fuel, pressure=gas.P,
                time_limit=time_limit,
                table_id=None
            )
    
    bdf_results[result_name] = results
    

Running cvode_bdf-None with rtol=1e-12 and atol=1e-10:  34%|███▍      | 0.027186000000006535/0.08 [01:42<04:21, 4955.77s/it, step=27186, temperature=813.6K, cpu_time=2.30e-03s, total_cpu_time=5.79e+01s]

In [ ]:
solvers_results_high = {}
for table_id in solver_to_run:
    if table_id in implicit_solvers:
        method = 'arkode_dirk'
    else:
        method = 'arkode_erk'
    gas, y0 = reset_gas(temperature, pressure, phi)
    rtol, atol = 1e-8, 1e-10

    results = run_integration_experiment(
                method, gas, y0, t0, end_time, timestep,
                rtol, atol, species_to_track,
                fuel, pressure=gas.P,
                time_limit=time_limit,
                table_id=table_id
            )
    solvers_results_high[table_id] = results


In [ ]:
solvers_results_low = {}
for table_id in solver_to_run:
    if table_id in implicit_solvers:
        method = 'arkode_dirk'
    else:
        method = 'arkode_erk'
    gas, y0 = reset_gas(temperature, pressure, phi)
    rtol, atol = 1e-6, 1e-8

    results = run_integration_experiment(
                method, gas, y0, t0, end_time, timestep,
                rtol, atol, species_to_track,
                fuel, pressure=gas.P,
                time_limit=time_limit,
                table_id=table_id
            )
    solvers_results_low[table_id] = results

In [ ]:
# PLOT THE CPU TIME OF THE IMPLICIT SOLVERS IN A 2x2 GRID
line_styles = ['-', '--', '-.', ':'] * 10  # Repeat basic line styles
colors = [ 'purple', 'orange', 'brown', 'pink',  'cyan', 'magenta', 'lime', 'teal', 'navy', 'gray', 'olive','maroon', 'gold', 'silver', 'indigo', 'turquoise']

# Create figure with 2x2 subplots sharing x and y axes
fig, ax = plt.subplots(figsize=(15, 10), dpi=200)

# Plot each group in its own subplot
for i, table_id in enumerate(solver_to_run):
    # if table_id in solver_to_exclude:
    #     continue
    data = solvers_results_high[table_id]
    if len(data['cpu_times']) < 100:
        continue
    ax.plot(
        running_average_forward(data['cpu_times'], 1000),
        label=f"{str(table_id)} high - {np.sum(data['cpu_times']):.2e}",
        linestyle='--',
        linewidth=2,
        color=colors[i]
    )
    data = solvers_results_low[table_id]
    ax.plot(
        running_average_forward(data['cpu_times'], 1000),
        label=f"{str(table_id)} low - {np.sum(data['cpu_times']):.2e}",
        linestyle='-',
        linewidth=2,
        color=colors[i]
    )
ax.plot(
    running_average_forward(bdf_results['reference']['cpu_times'], 1000    ),
    label=f'Reference(1e-12,1e-10) - {np.sum(bdf_results['reference']['cpu_times']):.2e}',
    linestyle='-',
    linewidth=3,
    color='red'
)

ax.plot(
    running_average_forward(bdf_results['bdf_results_low']['cpu_times'], 1000),
    label=f'BDF(1e-6,1e-8) - {np.sum(bdf_results['bdf_results_low']['cpu_times']):.2e}',
    linestyle='-',
    linewidth=2,
    color='blue'
)

ax.plot(
    running_average_forward(bdf_results['bdf_results_high']['cpu_times'], 1000),
    label=f'BDF(1e-8,1e-10) - {np.sum(bdf_results['bdf_results_high']['cpu_times']):.2e}',
    linestyle='-',
    linewidth=2,
    color='green'
)
# ax.set_title(f'Solvers {subplot_idx*6 + 1}-{min((subplot_idx+1)*6, 25)}')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=14)

# # Set common labels
# fig.text(0.5, 0.04, 'Step Number', ha='center', va='center')
# fig.text(0.06, 0.5, 'CPU Time (s)', ha='center', va='center', rotation='vertical')
fig.suptitle(f'CPU Time per Step for Different Implicit Solvers - {temperature}K - {pressure}Pa', fontsize=16)

plt.tight_layout()
plt.show()


In [ ]:
# PLOT THE CPU TIME OF THE IMPLICIT SOLVERS IN A 2x2 GRID
line_styles = ['-', '--', '-.', ':'] * 10  # Repeat basic line styles
colors = [ 'purple', 'orange', 'brown', 'pink',  'cyan', 'magenta', 'lime', 'teal', 'navy', 'gray', 'olive','maroon', 'gold', 'silver', 'indigo', 'turquoise']

# Create figure with 2x2 subplots sharing x and y axes
fig, ax = plt.subplots(figsize=(15, 10), dpi=200)

# Get reference data
reference_times = running_average_forward(bdf_results['reference']['cpu_times'], 1000)

# Plot each group in its own subplot
for i, table_id in enumerate(solver_to_run):
    # High tolerance results
    data = solvers_results_high[table_id]
    solver_times = running_average_forward(data['cpu_times'], 1000)
    if len(solver_times) < 100:
        continue
    ratio = solver_times / reference_times[:len(solver_times)]
    ax.plot(
        ratio,
        label=f"{str(table_id)} high - {np.mean(ratio):.2e}",
        linestyle='--',
        linewidth=2,
        color=colors[i]
    )
    
    # Low tolerance results
    data = solvers_results_low[table_id]
    solver_times = running_average_forward(data['cpu_times'], 1000)
    ratio = solver_times / reference_times[:len(solver_times)]
    ax.plot(
        ratio,
        label=f"{str(table_id)} low - {np.mean(ratio):.2e}",
        linestyle='-',
        linewidth=2,
        color=colors[i]
    )

# Plot BDF ratios
bdf_low_times = running_average_forward(bdf_results['bdf_results_low']['cpu_times'], 1000)
ratio = bdf_low_times / reference_times[:len(bdf_low_times)]
ax.plot(
    ratio,
    label=f'BDF(1e-6,1e-8) - {np.mean(ratio):.2e}',
    linestyle='-',
    linewidth=2,
    color='blue'
)

bdf_high_times = running_average_forward(bdf_results['bdf_results_high']['cpu_times'], 1000)
ratio = bdf_high_times / reference_times[:len(bdf_high_times)]
ax.plot(
    ratio,
    label=f'BDF(1e-8,1e-10) - {np.mean(ratio):.2e}',
    linestyle='-',
    linewidth=2,
    color='green'
)

# Plot reference ratio (always 1.0)
ax.plot(
    np.ones_like(reference_times),
    label='Reference(1e-12,1e-10)',
    linestyle='-',
    linewidth=3,
    color='red'
)

ax.set_yscale('log')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=14)
ax.set_ylabel('Ratio to Reference CPU Time', fontsize=14)
ax.set_xlabel('Step Number (thousands)', fontsize=14)
ax.grid(True)

fig.suptitle(f'CPU Time Ratio per Step for Different Implicit Solvers - {temperature}K - {pressure}Pa', fontsize=16)

plt.tight_layout()
plt.show()

In [ ]:
# Plot relative CPU times compared to reference integrator
colors = ['purple', 'orange', 'brown', 'pink', 'cyan', 'magenta', 'lime', 'teal', 'navy', 'gray', 'olive', 'maroon', 'gold', 'silver', 'indigo', 'turquoise']

# Create figure
fig, ax = plt.subplots(figsize=(15, 10), dpi=200)

reference_time = np.sum(bdf_results['reference']['cpu_times'])

# Plot BDF results
bdf_low_relative = np.sum(bdf_results['bdf_results_low']['cpu_times']) / reference_time
bdf_high_relative = np.sum(bdf_results['bdf_results_high']['cpu_times']) / reference_time

ax.scatter(['BDF(1e-6,1e-8)', 'BDF(1e-8,1e-10)'], 
          [bdf_low_relative, bdf_high_relative],
          s=200, color=['blue', 'green'], label='BDF')

# Plot other solvers
for i, table_id in enumerate(solver_to_run):
    if len(solvers_results_high[table_id]['cpu_times']) < 100:
        continue
    high_time = np.sum(solvers_results_high[table_id]['cpu_times']) / reference_time
    low_time = np.sum(solvers_results_low[table_id]['cpu_times']) / reference_time
    
    label = str(table_id).split('.')[-1]  # Get just the solver name
    ax.scatter([f'{label} high', f'{label} low'],
              [high_time, low_time], 
              s=200,
              color=colors[i],
              label=label)

ax.axhline(y=1.0, color='red', linestyle='--', label='Reference')
ax.set_ylabel('Relative CPU Time (vs Reference)', fontsize=14)
ax.set_xlabel('Solver Configuration', fontsize=14)
ax.set_yscale('log')
ax.grid(True)
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=14)

plt.xticks(rotation=45, ha='right')
fig.suptitle(f'Relative CPU Time for Different Implicit Solvers - {temperature}K - {pressure}Pa', fontsize=16)

plt.tight_layout()
plt.show()


In [ ]:
solver_errors = {}
solver_errors_high = {}
explicit_solver_errors = {}
species_to_track = ['temperature', 'h', 'h2', 'o', 'o2', 'h2o', 'ho2', 'h2o2', 'oh']
for table_id in solver_to_run:
    if len(solvers_results_low[table_id]['cpu_times']) < 100:
        continue
    solver_errors[table_id] = calculate_rmse(bdf_results['reference'], solvers_results_low[table_id], species_to_track, use_log=False)
    solver_errors_high[table_id] = calculate_rmse(bdf_results['reference'], solvers_results_high[table_id], species_to_track, use_log=False)


bdf_results_errors = calculate_rmse(bdf_results['reference'], bdf_results['bdf_results_low'], species_to_track, use_log=False)

bdf_results_errors_high = calculate_rmse(bdf_results['reference'], bdf_results['bdf_results_high'], species_to_track, use_log=False)



In [ ]:
solver_to_exclude = [SP.arkode.ButcherTable.HEUN_EULER_2_1_2 , SP.arkode.ButcherTable.ARK324L2SA_ERK_4_2_3, SP.arkode.ButcherTable.ARK437L2SA_DIRK_7_3_4, SP.arkode.ButcherTable.BOGACKI_SHAMPINE_4_2_3, SP.arkode.ButcherTable.CASH_5_3_4, SP.arkode.ButcherTable.HEUN_EULER_2_1_2]


In [ ]:
solver_to_plot = [SP.arkode.ButcherTable.HEUN_EULER_2_1_2 , SP.arkode.ButcherTable.BOGACKI_SHAMPINE_4_2_3, SP.arkode.ButcherTable.ARK2_DIRK_3_1_2, SP.arkode.ButcherTable.TRBDF2_3_3_2]

In [ ]:
# Create one large figure with subplots for all species
fig, axes = plt.subplots(3, 3, figsize=(30, 30), dpi=300)

for i, specie_name in enumerate(species_to_track):
    ax = axes[i // 3, i % 3]
    ax.plot(running_average_forward(np.maximum(bdf_results_errors_high[specie_name], 1e-20), 1000),
            label=f"BDF(1e-8,1e-10)", 
            linestyle='-',
            linewidth=2,
                color='blue')
    for j, table_id in enumerate(solver_to_plot):
        if len(solver_errors_high[table_id][specie_name]) < 100:
            continue
        data = solver_errors_high[table_id]
        ax.plot(running_average_forward(np.maximum(data[specie_name], 1e-20), 1000),
            label=f"{str(table_id)}", 
            linestyle='--',
            linewidth=2,
                color=colors[i])
        
        data = solver_errors[table_id]
        ax.plot(running_average_forward(np.maximum(data[specie_name], 1e-20), 1000),
            label=f"{str(table_id)}", 
            linestyle='-',
            linewidth=2,
                color=colors[i])
        
    ax.plot(running_average_forward(np.maximum(bdf_results_errors[specie_name], 1e-20), 1000),
            label=f"BDF(1e-6,1e-8)", 
            linestyle='-',
            linewidth=2,
                color='red')
    
    ax.set_title(f'{specie_name}')
    ax.set_yscale('log')
    ax.grid(True)
    if i == len(species_to_track) - 1:  # Only show legend on last plot
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=15)

fig.suptitle('RMSE for Different Species Across All Implicit Solvers', fontsize=16)
plt.tight_layout()
plt.show()


In [ ]:
solver_to_run = solver_to_plot

In [ ]:
# PLOT THE CPU TIME OF THE IMPLICIT SOLVERS IN A 2x2 GRID
line_styles = ['-', '--', '-.', ':'] * 10  # Repeat basic line styles
colors = [ 'purple', 'orange', 'brown', 'pink',  'cyan', 'magenta', 'lime', 'teal', 'navy', 'gray', 'olive','maroon', 'gold', 'silver', 'indigo', 'turquoise']

# Create figure with 2x2 subplots sharing x and y axes
fig, ax = plt.subplots(figsize=(15, 10), dpi=200)

# Plot each group in its own subplot
for i, table_id in enumerate(solver_to_run):
    # if table_id in solver_to_exclude:
    #     continue
    data = solvers_results_high[table_id]
    ax.plot(
        data['temperatures'],
        label=f"{str(table_id)} high - {np.sum(data['cpu_times']):.2e}",
        linestyle='--',
        linewidth=2,
        color=colors[i]
    )
    data = solvers_results_low[table_id]
    ax.plot(
        data['temperatures'],
        label=f"{str(table_id)} low - {np.sum(data['cpu_times']):.2e}",
        linestyle='-',
        linewidth=2,
        color=colors[i]
    )
ax.plot(
    bdf_results['reference']['temperatures'],
    label=f'Reference(1e-12,1e-10) - {np.sum(bdf_results['reference']['cpu_times']):.2e}',
    linestyle='-',
    linewidth=3,
    color='red'
)

ax.plot(
    bdf_results['bdf_results_low']['temperatures'],
    label=f'BDF(1e-6,1e-8) - {np.sum(bdf_results['bdf_results_low']['cpu_times']):.2e}',
    linestyle='-',
    linewidth=2,
    color='blue'
)

ax.plot(
    bdf_results['bdf_results_high']['temperatures'],
    label=f'BDF(1e-8,1e-10) - {np.sum(bdf_results['bdf_results_high']['cpu_times']):.2e}',
    linestyle='-',
    linewidth=2,
    color='green'
)
# ax.set_title(f'Solvers {subplot_idx*6 + 1}-{min((subplot_idx+1)*6, 25)}')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=14)

# # Set common labels
# fig.text(0.5, 0.04, 'Step Number', ha='center', va='center')
# fig.text(0.06, 0.5, 'CPU Time (s)', ha='center', va='center', rotation='vertical')
fig.suptitle(f'Temperature Profile for Different Implicit Solvers - {temperature}K - {pressure}Pa', fontsize=16)

plt.tight_layout()
plt.show()


In [ ]:
def calculate_maximum_species_conc(data, ref_data, species_name):
    max_concentrations = {}
    for specie in species_name:
        if specie == 'temperature':
            max_conc = np.mean(data['temperatures'])
            ref_max_conc = np.mean(ref_data['temperatures'])
        else:
            max_conc = np.mean(data['species_profiles'][specie])
            ref_max_conc = np.mean(ref_data['species_profiles'][specie])
        max_concentrations[specie] = (max_conc, ref_max_conc)
    return max_concentrations

solver_max_conc = {}
solver_max_conc_high = {}

species_to_track = ['temperature', 'h', 'h2', 'o', 'o2', 'h2o', 'ho2', 'h2o2', 'oh']
for table_id in solver_to_run:
    if len(solvers_results_low[table_id]['cpu_times']) < 100:
        continue
    solver_max_conc[table_id] = calculate_maximum_species_conc(solvers_results_low[table_id], bdf_results['reference'], species_to_track)
    solver_max_conc_high[table_id] = calculate_maximum_species_conc(solvers_results_high[table_id], bdf_results['reference'], species_to_track)


bdf_max_conc = calculate_maximum_species_conc(bdf_results['reference'], bdf_results['bdf_results_low'], species_to_track)

bdf_max_conc_high = calculate_maximum_species_conc(bdf_results['reference'], bdf_results['bdf_results_high'], species_to_track)





In [ ]:
# Create a figure for plotting max concentration ratios
fig, ax = plt.subplots(figsize=(30, 20), dpi=300)

# Track x-axis positions and labels
x_positions = []
x_labels = []
current_x = 0
colors = ['blue', 'green', 'orange', 'purple', 'brown', 'pink', 'cyan', 'magenta', 'lime', 'teal', 'navy', 'gray', 'olive', 'maroon', 'gold', 'silver', 'indigo', 'turquoise']
# Plot for each species
for specie_name in species_to_track:
    # Plot BDF results
    bdf_ratio_low = bdf_max_conc[specie_name][0] / bdf_max_conc[specie_name][1]
    bdf_ratio_high = bdf_max_conc_high[specie_name][0] / bdf_max_conc_high[specie_name][1]
    
    ax.scatter(current_x, bdf_ratio_low, color='red', s=200, label='BDF (1e-6,1e-8)' if specie_name == species_to_track[0] else "")
    ax.scatter(current_x, bdf_ratio_high, color='black', s=200, label='BDF (1e-8,1e-10)' if specie_name == species_to_track[0] else "")
    
    # Plot other solver results
    for i, table_id in enumerate(solver_to_run):
        if len(solvers_results_high[table_id]['cpu_times']) < 100:
            print(f"Skipping {table_id} because it has less than 100 steps")
            continue
        ratio_low = solver_max_conc[table_id][specie_name][0] / solver_max_conc[table_id][specie_name][1]
        ratio_high = solver_max_conc_high[table_id][specie_name][0] / solver_max_conc_high[table_id][specie_name][1]
        
        ax.scatter(current_x, ratio_low, color=colors[i], s=200, 
                  label=f"{str(table_id)} (1e-6,1e-8)" if specie_name == species_to_track[0] else "")
        ax.scatter(current_x, ratio_high, color=colors[i], s=200, marker='x',
                  label=f"{str(table_id)} (1e-8,1e-10)" if specie_name == species_to_track[0] else "")
    
    x_positions.append(current_x)
    x_labels.append(specie_name)
    current_x += 1

# Customize plot
ax.set_xticks(x_positions)
ax.set_xticklabels(x_labels, rotation=45, fontsize=16)
ax.set_yscale('log')
ax.grid(True)
ax.set_ylabel('Max Concentration Ratio (Solver/Reference)', fontsize=16)
ax.set_title(f'Maximum Concentration Ratios for Different Species and Solvers - {temperature}K - {pressure}Pa', fontsize=16)
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=16)
plt.tight_layout()
plt.show()
